In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 读取数据
df = pd.read_csv('output.csv')

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# 将数据分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 转换为Tensor
train_features = torch.tensor(X_train, dtype=torch.float32)
train_targets = torch.tensor(y_train, dtype=torch.float32)
test_features = torch.tensor(X_test, dtype=torch.float32)
test_targets = torch.tensor(y_test, dtype=torch.float32)

# 创建DataLoader
train_dataset = TensorDataset(train_features, train_targets)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 初始化模型、损失函数和优化器
model = SimpleNet(input_size=X_train.shape[1], hidden_size=50, num_classes=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 保存训练的模型
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
}
torch.save(checkpoint, 'model_checkpoint.pth')

model.eval()
with torch.no_grad():
    test_outputs = model(test_features) 
    
test_outputs = test_outputs.cpu().numpy()
test_targets = test_targets.cpu().numpy()

# 打印十个样本的实际值和预测值
for i in range(10):
    print(f"Sample {i+1}:")
    print(f"Actual Value: {test_targets[i]}")
    print(f"Predicted Value: {test_outputs[i][0]}")
    print("-" * 30) # 字符串乘法，太有Python了



c:\Users\hghgh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\hghgh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([14])) that is different to the input size (torch.Size([14, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Sample 1:
Actual Value: 10.0
Predicted Value: 10.58698844909668
------------------------------
Sample 2:
Actual Value: 15.0
Predicted Value: 8.483835220336914
------------------------------
Sample 3:
Actual Value: 15.0
Predicted Value: 9.075948715209961
------------------------------
Sample 4:
Actual Value: 11.0
Predicted Value: 8.304685592651367
------------------------------
Sample 5:
Actual Value: 9.0
Predicted Value: 9.347143173217773
------------------------------
Sample 6:
Actual Value: 6.0
Predicted Value: 8.912187576293945
------------------------------
Sample 7:
Actual Value: 19.0
Predicted Value: 9.629240036010742
------------------------------
Sample 8:
Actual Value: 0.0
Predicted Value: 12.05241584777832
------------------------------
Sample 9:
Actual Value: 4.0
Predicted Value: 9.605371475219727
------------------------------
Sample 10:
Actual Value: 20.0
Predicted Value: 10.259649276733398
------------------------------
